In [2]:
# A. 모델 성능 / 스크리닝 뷰 (exp01, exp02만)
import json, pandas as pd
from glob import glob

rows = []

for f in glob("../results/**/*_result.json"):
    with open(f, encoding="utf-8") as fp:
        data = json.load(fp)

    # 정책 매트릭스 제외
    if data["experiment_id"] == "exp03_policy_matrix":
        continue

    rows.append({
        "experiment_id": data["experiment_id"],
        "persona": data["persona"],
        "roc_auc": data.get("roc_auc"),
        "policy_message": data["policy_message"]
    })

pd.DataFrame(rows)


,experiment_id,persona,roc_auc,policy_message
0,exp01_disconnect_risk,구조적 단절 위험 청년,0.6851,단절위험 환경에 놓인 청년은 개인 역량 강화만으로는 재진입이 어렵다. 구조적 개입 필요.
1,exp02_employment_prediction,훈련 반복형 / 전공불일치 청년,0.7486,훈련 투입 전에 개인의 구조적 위치(연령×학력×성별)를 먼저 파악해야 효과적


In [ ]:
#정책 설계 뷰 (exp03 전용)
import json

with open("../results/oxju/exp03_policy_matrix.json", encoding="utf-8") as f:
     policy = json.load(f)

summary = {
    "experiment_id": policy["meta"]["experiment_id"],
    "persona": policy["persona"],
    "hypothesis": policy["hypothesis"],
    "total_n": policy["metrics"]["총_표본수"],
    "고착형_비율": policy["metrics"]["고착형_비율"],
    "재연결_비율": policy["metrics"]["재연결_비율"],
    "취업취약_비율": policy["metrics"]["취업취약_비율"],
    "안정군_비율": policy["metrics"]["안정군_비율"],
    "policy_message": policy["results"]["policy_message"]
}

df_policy_overview = pd.DataFrame([summary])
df_policy_overview


,취업가능성 ↓ / 단절위험 →,단절위험 높음,단절위험 낮음
0,고착형 단절 위험군,고착형 단절 위험군\n(재참여·구조개입),
1,재연결 가능 단절군,재연결 가능 단절군\n(훈련+채용),
2,취업취약 비단절군,,취업취약 비단절군\n(방향 교정)
3,안정군,,안정군\n(모니터링)


In [9]:
# 정책 매트릭스 2×2 구조 표
matrix_table = pd.DataFrame({
    "취업가능성 ↓ / 단절위험 →": [
        "고착형 단절 위험군",
        "재연결 가능 단절군",
        "취업취약 비단절군",
        "안정군"
    ],
    "단절위험 높음": [
        "고착형 단절 위험군\n(재참여·구조개입)",
        "재연결 가능 단절군\n(훈련+채용)",
        "",
        ""
    ],
    "단절위험 낮음": [
        "",
        "",
        "취업취약 비단절군\n(방향 교정)",
        "안정군\n(모니터링)"
    ]
})

matrix_table


,취업가능성 ↓ / 단절위험 →,단절위험 높음,단절위험 낮음
0,고착형 단절 위험군,고착형 단절 위험군\n(재참여·구조개입),
1,재연결 가능 단절군,재연결 가능 단절군\n(훈련+채용),
2,취업취약 비단절군,,취업취약 비단절군\n(방향 교정)
3,안정군,,안정군\n(모니터링)


In [10]:
# 분포 수치 포함한 “정책 매트릭스 표”
dist = policy["results"]["key_notes"]["정책유형_분포"]

matrix_numeric = pd.DataFrame({
    "정책유형": [
        "고착형 단절 위험군",
        "재연결 가능 단절군",
        "취업취약 비단절군",
        "안정군"
    ],
    "인원수": [
        dist["고착형 단절 위험군"],
        dist["재연결 가능 단절군"],
        dist["취업취약 비단절군"],
        dist["안정군"]
    ],
    "비율": [
        policy["metrics"]["고착형_비율"],
        policy["metrics"]["재연결_비율"],
        policy["metrics"]["취업취약_비율"],
        policy["metrics"]["안정군_비율"]
    ]
})

matrix_numeric


,정책유형,인원수,비율
0,고착형 단절 위험군,2684,0.215
1,재연결 가능 단절군,2327,0.186
2,취업취약 비단절군,1670,0.134
3,안정군,5800,0.465


본 정책 매트릭스는 단절위험(환경)과 취업가능성(개인)을
서로 독립적인 두 축으로 분리하여 청년을 4개 정책유형으로 분류한다.

이는 ‘누가 취업할 것인가’를 맞히기 위한 예측 모델이 아니라,
정책 개입이 필요한 대상을 구조적으로 선별하기 위한 설계 도구이다.

특히 고착형 단절 위험군은 전체의 약 21.5%로,
단순 훈련 제공보다는 재참여 유도 및 구조적 개입이 우선적으로 요구된다.